In [41]:
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
import random
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
from scipy import sparse as ssp
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [42]:
df_train=pd.read_csv("train.csv")

In [43]:
abhi_test=pd.read_csv("../input/rader/test_features.csv")
abhi_train=pd.read_csv("../input/rader/train_features.csv")
base_test=pd.read_csv("../input/rader/test_base_features.csv")
base_train=pd.read_csv("../input/rader/train_base_features.csv")

abhi_test=abhi_test.drop(['question1','question2'],axis=1)
abhi_train=abhi_train.drop(['question1','question2'],axis=1)
#base_test=base_test.drop(['question1','question2'],axis=1)
#base_train=base_train.drop(['question1','question2'],axis=1)
features=list(set(abhi_train.columns.tolist()).union(base_train.columns.tolist()))

In [44]:
train=pd.concat([abhi_train,base_train],axis=1)
test=pd.concat([abhi_test,base_test],axis=1)

In [45]:
features=list(set(abhi_train.columns.tolist()).union(base_train.columns.tolist()))

In [46]:
import lightgbm as lgbm
def runlgbm(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=500000,e_stoping_r=50): 
    t4_params = {
        'boosting_type': 'gbdt', 'objective': 'binary', 'nthread': -1, 'silent': True,
        'num_leaves': 6, 'learning_rate': 0.05, 'max_depth': 6,
        'max_bin': 255, 'subsample_for_bin': 50000,
        'subsample': 0.4, 'subsample_freq': 1, 'colsample_bytree': 0.4, 'reg_alpha':1, 'reg_lambda':0,
        'min_split_gain': 0.5, 'min_child_weight': 1, 'min_child_samples': 10, 'scale_pos_weight': 1}

    # they can be used directly to build a LGBMClassifier (which is wrapped in a sklearn fashion)
    model = lgbm.sklearn.LGBMClassifier(n_estimators=num_rounds, seed=0, **t4_params)
    
    if test_y is not None:
        model.fit(train_X,train_y,eval_set=[(train_X,train_y),(test_X, test_y)],verbose=50,early_stopping_rounds=e_stoping_r)
    else:
        model.fit(train_X,train_y)
    pred_test_y = model.predict_proba(test_X)
    return pred_test_y, model

In [47]:

#features=train.columns.tolist()
#features.remove("question1")
#features.remove("question2")
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=321, num_rounds=2000):
    param = {}
    param['objective'] = 'binary:logistic'
    param['eta'] = 0.03
    param['max_depth'] = 6
    param['silent'] = 1
    param['eval_metric'] = "logloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.3
    param['colsample_bytree'] = 0.3
    param['seed'] = seed_val
    #param['lambda'] = 0.005
    param['alpha'] = 0.005
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20,verbose_eval=50)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [57]:

path="/home/udit/ipython/notebook/quora/input/input/"
#train = pd.read_csv(path+"train_porter.csv")    

train_question1_tfidf = pd.read_pickle(path+'train_question1_tfidf.pkl')
train_porter_interaction = pd.read_pickle(path+'train_porter_interaction.pkl')

train_question1_tfidf = pd.read_pickle(path+'train_question1_tfidf.pkl')[:]
test_question1_tfidf = pd.read_pickle(path+'test_question1_tfidf.pkl')[:]


train_question2_tfidf = pd.read_pickle(path+'train_question2_tfidf.pkl')[:]
test_question2_tfidf = pd.read_pickle(path+'test_question2_tfidf.pkl')[:]

train_interaction = pd.read_pickle(path+'train_interaction.pkl')[:].reshape(-1,1)
test_interaction = pd.read_pickle(path+'test_interaction.pkl')[:].reshape(-1,1)

train_interaction=np.nan_to_num(train_interaction)
test_interaction=np.nan_to_num(test_interaction)      


train_porter_interaction = pd.read_pickle(path+'train_porter_interaction.pkl')[:].reshape(-1,1)
test_porter_interaction = pd.read_pickle(path+'test_porter_interaction.pkl')[:].reshape(-1,1)


train_porter_interaction=np.nan_to_num(train_porter_interaction)
test_porter_interaction=np.nan_to_num(test_porter_interaction)


train_jaccard = pd.read_pickle(path+'train_jaccard.pkl')[:].reshape(-1,1)
test_jaccard = pd.read_pickle(path+'test_jaccard.pkl')[:].reshape(-1,1)

train_jaccard=np.nan_to_num(train_jaccard)
test_jaccard=np.nan_to_num(test_jaccard)

train_porter_jaccard = pd.read_pickle(path+'train_porter_jaccard.pkl')[:].reshape(-1,1)
test_porter_jaccard = pd.read_pickle(path+'test_porter_jaccard.pkl')[:].reshape(-1,1)


train_jaccard=np.nan_to_num(train_jaccard)
test_porter_jaccard=np.nan_to_num(test_porter_jaccard)

train_len = pd.read_pickle(path+"train_len.pkl")
test_len = pd.read_pickle(path+"test_len.pkl")

train_len=np.nan_to_num(train_len)
test_len=np.nan_to_num(test_len) 

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:15: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:16: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:23: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:31: FutureWarning: reshape is

In [49]:
train[features].shape

(404290, 48)

In [60]:
#del df_train
#del abhi_test
#del base_test
#del base_train
#del abhi_train
#import gc
#gc.collect()
from scipy import sparse as ssp
X=ssp.csr_matrix(train[features])
X_test=ssp.csr_matrix(test[features])

In [58]:
scaler = MinMaxScaler()
scaler.fit(np.vstack([train_len,test_len]))
train_len = scaler.transform(train_len)
test_len =scaler.transform(test_len)

In [61]:
X_tr = ssp.hstack([
    X,
    train_question1_tfidf,
    train_question2_tfidf,
    train_interaction,
    train_porter_interaction,
    train_jaccard,
    train_porter_jaccard,
    train_len
    ]).tocsr()

del X

In [62]:
df_train=pd.read_csv("train.csv",usecols=['id','is_duplicate'])
y =df_train['is_duplicate'].values[:]

X_t = ssp.hstack([
    X_test,
    test_question1_tfidf,
    test_question2_tfidf,
    test_interaction,
    test_porter_interaction,
    test_jaccard,
    test_porter_jaccard,
    test_len
    ]).tocsr()

del X_test

In [63]:
y =df_train['is_duplicate'].values[:]
X_train, X_test, y_train, y_test = train_test_split(X_tr, y, test_size=0.20, random_state=42)
preds, model = runXGB(X_tr, y,X_t,num_rounds=700)

In [65]:
test=pd.read_csv("test.csv",usecols=['test_id'])

In [66]:
test.loc[:,"is_duplicate"]=preds

In [67]:
test.to_csv("only_xgb.csv",index=False)

In [64]:
def fromsparsetofile(filename, array, deli1=" ", deli2=":",ytarget=None):    
    zsparse=csr_matrix(array)
    indptr = zsparse.indptr
    indices = zsparse.indices
    data = zsparse.data
    print(" data lenth %d" % (len(data)))
    print(" indices lenth %d" % (len(indices)))    
    print(" indptr lenth %d" % (len(indptr)))
    
    f=open(filename,"w")
    counter_row=0
    for b in range(0,len(indptr)-1):
        #if there is a target, print it else , print nothing
        if ytarget!=None:
             f.write(str(ytarget[b]) + deli1)     
             
        for k in range(indptr[b],indptr[b+1]):
            if (k==indptr[b]):
                if np.isnan(data[k]):
                    f.write("%d%s%f" % (indices[k],deli2,-1))
                else :
                    f.write("%d%s%f" % (indices[k],deli2,data[k]))                    
            else :
                if np.isnan(data[k]):
                     f.write("%s%d%s%f" % (deli1,indices[k],deli2,-1))  
                else :
                    f.write("%s%d%s%f" % (deli1,indices[k],deli2,data[k]))
        f.write("\n")
        counter_row+=1
        if counter_row%10000==0:    
            print(" row : %d " % (counter_row))    
    f.close()  

In [67]:
from scipy.sparse import csr_matrix,hstack
fromsparsetofile("/home/udit/ipython/notebook/quora/input/input/all_features/train_intraction.sparse", X_tr, deli1=" ", deli2=":",ytarget=y)    
fromsparsetofile("/home/udit/ipython/notebook/quora/input/input/all_features/test_intraction.sparse", X_t, deli1=" ", deli2=":",ytarget=None) 

 data lenth 38582281
 indices lenth 38582281
 indptr lenth 404291


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


 row : 10000 
 row : 20000 
 row : 30000 
 row : 40000 
 row : 50000 
 row : 60000 
 row : 70000 
 row : 80000 
 row : 90000 
 row : 100000 
 row : 110000 
 row : 120000 
 row : 130000 
 row : 140000 
 row : 150000 
 row : 160000 
 row : 170000 
 row : 180000 
 row : 190000 
 row : 200000 
 row : 210000 
 row : 220000 
 row : 230000 
 row : 240000 
 row : 250000 
 row : 260000 
 row : 270000 
 row : 280000 
 row : 290000 
 row : 300000 
 row : 310000 
 row : 320000 
 row : 330000 
 row : 340000 
 row : 350000 
 row : 360000 
 row : 370000 
 row : 380000 
 row : 390000 
 row : 400000 
 data lenth 224877639
 indices lenth 224877639
 indptr lenth 2345797
 row : 10000 
 row : 20000 
 row : 30000 
 row : 40000 
 row : 50000 
 row : 60000 
 row : 70000 
 row : 80000 
 row : 90000 
 row : 100000 
 row : 110000 
 row : 120000 
 row : 130000 
 row : 140000 
 row : 150000 
 row : 160000 
 row : 170000 
 row : 180000 
 row : 190000 
 row : 200000 
 row : 210000 
 row : 220000 
 row : 230000 
 row

In [68]:
np.savez("/home/udit/ipython/notebook/quora/input/input/all_features/trainmatrix", data=X_tr.data, indices=X_tr.indices,indptr=X_tr.indptr, shape=X_tr.shape)
np.savez("/home/udit/ipython/notebook/quora/input/input/all_features/testmatrix", data=X_t.data, indices=X_tr.indices,indptr=X_tr.indptr, shape=X_tr.shape)

In [14]:
path="/home/udit/ipython/notebook/quora/input/input/all_features/"
from scipy.sparse import csr_matrix
loader = np.load(path+"trainmatrix.npz")
X_tr=csr_matrix((loader['data'], loader['indices'], loader['indptr']),shape=loader['shape'])

In [8]:
X_tr=csr_matrix((loader['data'], loader['indices'], loader['indptr']),shape=loader['shape'])

In [16]:
loader = np.load(path+"testmatrix.npz")
#X_t=csr_matrix((loader2['data'], loader2['indices'], loader2['indptr']),shape=loader2['shape'])

In [17]:
len(loader['indices']),len(loader['data'])

(38582281, 224877639)